In [1]:
import numpy as np
import pandas as pd

import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import timm

import random
import os
from astropy.io import fits
from astropy.table import Table

import matplotlib.pyplot as plt
from astropy.visualization import make_lupton_rgb
plt.style.use('dark_background')

In [2]:
def make_plot_all(objects, data, Q, stretch, minimum):
    for i in range(len(objects)):
        if(i%8 == 0):
            plt.figure(figsize=(16,9))
            for j in range(8):
                if(i+j > len(objects)-1): break
                plt.subplot(1,8,j+1)
                #print(data['new_id'].iloc[i+j])
                title = data['new_id'].iloc[i+j]
                plt.title(title)
                rgb = make_lupton_rgb(objects[i+j][2], objects[i+j][1], objects[i+j][0], 
                                    Q=Q, stretch=stretch, minimum=minimum)
                plt.imshow(rgb, aspect='equal')
                plt.xticks([], [])
                plt.yticks([], []) 
            plt.show()
            
def make_one_plot(objects, title, data, Q, stretch, minimum):
    plt.figure(figsize=(4,4))
    rgb = make_lupton_rgb(objects[2], objects[1], objects[0], Q=Q, stretch=stretch, minimum=minimum)
    rgb = np.flipud(rgb)
    plt.imshow(rgb, aspect='equal')
    plt.xticks([], [])
    plt.yticks([], []) 
    plt.savefig(png_path + title, bbox_inches='tight')
    #plt.show()
    plt.close()
    
def make_one_plot_brit(objects, title, data):
    plt.figure(figsize=(4,4))
    
    imager = 0.0005*objects[2] 
    imageg = 3*objects[1] 
    imageb = 15*objects[0] 
    rgb = np.array([imager, imageg, imageb])
    #Making sure values are lower or equal to 255
    rgb = np.clip(rgb, 0, 255).astype(np.uint8)
    rgb = rgb.transpose(1, 2, 0)
    #Increasing contrast
    min_img, max_img = np.min(rgb), np.max(rgb)     
    LUT=np.zeros(256,dtype=np.uint8)
    LUT[min_img:max_img+1]=np.linspace(start=0,stop=255,num=(max_img-min_img)+1,endpoint=True,dtype=np.uint8)
    rgb = LUT[rgb]
    rgb = np.flipud(rgb)
                
    plt.imshow(rgb, aspect='equal')
    plt.xticks([], [])
    plt.yticks([], []) 
    
    plt.savefig(png_path + title, bbox_inches='tight')
    #plt.show()
    plt.close()

In [3]:
def write_fit_file(name, x, data):
    my_types = {'new_id': int, 'class': str}
    data = data.astype(my_types)
    
    primary = fits.PrimaryHDU()
    image = fits.ImageHDU(x, name="IMAGE")
    table_data = Table.from_pandas(data)
    table = fits.BinTableHDU(data = table_data)
    hdu_list = fits.HDUList([primary, image, table])
    hdu_list.writeto(name + '.fits', overwrite=True)   
    hdu_list.close()

In [4]:
# Saving all training images into one file (not including known strong lenses)
names = ['Single', 'Ring', 'Smooth', 'Companions', 'SDSS_Spirals', 'DES_Spirals', 'Crowded',
        'Most_negatives'] #'Double', #Single
file_path = '/Users/jimenagonzalez/research/DSPL/SpaceWarps_Inspection/training_sample/coadds/'
numpix = 45
images_save = np.zeros((1,3,numpix,numpix))
data_save = pd.DataFrame(columns = ['new_id', 'class'])

for k in range(len(names)):
    if(names[k] == 'DES_Spirals' or names[k] == 'SDSS_Spirals'): 
        num = 96 #96
    elif(names[k] == 'Single'):
        num = 855 #855
    else:
        num = 192 #192
        
    hdu_list = fits.open(file_path + 'new_ids_' + names[k] + '.fits')
    images = hdu_list[1].data
    data = pd.DataFrame(hdu_list[2].data)
    data = data.astype({'new_id': int})
    hdu_list.close()
    
    remove = pd.read_csv(file_path + 'remove_' + names[k] + '.csv', dtype = {'id': int})
    data = data.drop(remove['id'])
    
    images = images[data.index][0:num]
    data = data.iloc[0:num]
    group_list = [names[k]]*num
    data['class'] = group_list
    
    images_save = np.append(images_save, images, axis = 0)
    data_save = data_save.append(data)
    

images_save = np.delete(images_save, 0, axis = 0)
data_save = data_save.reset_index(drop=True)
print(len(images_save), len(data_save))

2007 2007


In [5]:
print(len(data_save))
data_save.tail()

2007


,new_id,class
2002,210,Most_negatives
2003,211,Most_negatives
2004,212,Most_negatives
2005,213,Most_negatives
2006,214,Most_negatives


In [6]:
feedback = pd.read_csv('Feedback.csv', sep='\t')

In [7]:
png_path = 'sample/'
sub_types = {'Single': 'Simulation', 'Ring': 'Ring', 'Smooth': 'Elliptical/Smooth', 'Companions': 'Neighbours/Companions', 
             'SDSS_Spirals': 'Spiral', 'DES_Spirals': 'Spiral', 'Crowded': 'Neighbours/CrowdedField',
        'Most_negatives': 'Isolated elliptical', 'Real': 'Known'}

# Saving each subset (manuscript, pngs) - not including real lenses:
for k in range(len(names)):
    data_class = pd.DataFrame()
    data_tmp = data_save[data_save['class'] == names[k]]
    images_tmp = images_save[data_tmp.index]
    feedback_tmp = feedback[feedback['Name'] == names[k]]
        
    data_class['!Subject Type'] = ['Training']*len(data_tmp)
    data_class['!Training SubType'] = [sub_types[names[k]]]*len(data_tmp)
    data_class['#CLASS'] = data_tmp['class'].apply(lambda x: f'train_{x}').values
    data_class['#ID'] = data_tmp['new_id'].values
    data_class['#IMAGE_NAME_1'] = data_tmp['new_id'].apply(lambda x: f'pngs/Train_{names[k]}_{x}_1.png').values
    data_class['#IMAGE_NAME_2'] = data_tmp['new_id'].apply(lambda x: f'pngs/Train_{names[k]}_{x}_2.png').values
    data_class['#IMAGE_NAME_3'] = data_tmp['new_id'].apply(lambda x: f'pngs/Train_{names[k]}_{x}_3.png').values
    data_class['#IMAGE_NAME_4'] = data_tmp['new_id'].apply(lambda x: f'pngs/Train_{names[k]}_{x}_4.png').values
    data_class['#feedback_1_x'] = [166]*len(data_tmp)
    data_class['#feedback_1_y'] = [166]*len(data_tmp)
    data_class['#feedback_1_tolerance'] = [150]*len(data_tmp)
    data_class['#feedback_1_successMessage'] = [feedback_tmp['Positive'].iloc[0]]*len(data_tmp)
    data_class['#feedback_1_failureMessage'] = [feedback_tmp['Negative'].iloc[0]]*len(data_tmp)
    data_class['#Provenance'] = ['DES-VT-JG']*len(data_tmp)
    data_class['Colours'] = ['gri']*len(data_tmp)
    data_class['Size-in-arcsec'] = [12]*len(data_tmp)
    
    if(names[k] == 'Single'):
        data_class['#Type'] = ['LENS']*len(data_tmp)
        data_class['!Training type'] = ['Positive']*len(data_tmp)
        data_class['#feedback_1_id'] = [1]*len(data_tmp)
    else:
        data_class['#Type'] = ['DUD']*len(data_tmp)
        data_class['!Training type'] = ['Negative']*len(data_tmp)
        data_class['#feedback_1_id'] = [2]*len(data_tmp)
    
    data_class['#feedback_1_type'] = data_class['#Type']
    data_class['#feedback_1_subtype'] = data_class['!Training SubType']
    
    #data_class.to_csv('sample/train_' + names[k] + '.csv', index=False)
    
    """
    for i in range(len(data_class)):
        title1 = str(data_class['#IMAGE_NAME_1'].iloc[i])
        make_one_plot(images_tmp[i], title1, data_class, 9, 60, (0,0,0)) 
        title2 = str(data_class['#IMAGE_NAME_2'].iloc[i])
        make_one_plot(images_tmp[i], title2, data_class, 8, 40, (14,2,0))
        title3 = str(data_class['#IMAGE_NAME_3'].iloc[i])
        make_one_plot(images_tmp[i], title3, data_class, 10, 37, (12,6,0))
        title4 = str(data_class['#IMAGE_NAME_4'].iloc[i])
        make_one_plot_brit(images_tmp[i], title4, data_class)
    """
    
data_class.head()

,!Subject Type,!Training SubType,#CLASS,#ID,#IMAGE_NAME_1,#IMAGE_NAME_2,#IMAGE_NAME_3,#IMAGE_NAME_4,#feedback_1_x,#feedback_1_y,...,#feedback_1_successMessage,#feedback_1_failureMessage,#Provenance,Colours,Size-in-arcsec,#Type,!Training type,#feedback_1_id,#feedback_1_type,#feedback_1_subtype
0,Training,Isolated elliptical,train_Most_negatives,0,pngs/Train_Most_negatives_0_1.png,pngs/Train_Most_negatives_0_2.png,pngs/Train_Most_negatives_0_3.png,pngs/Train_Most_negatives_0_4.png,166,166,...,Great job! You correctly identified that this ...,The feature you clicked on is not a lensed fea...,DES-VT-JG,gri,12,DUD,Negative,2,DUD,Isolated elliptical
1,Training,Isolated elliptical,train_Most_negatives,1,pngs/Train_Most_negatives_1_1.png,pngs/Train_Most_negatives_1_2.png,pngs/Train_Most_negatives_1_3.png,pngs/Train_Most_negatives_1_4.png,166,166,...,Great job! You correctly identified that this ...,The feature you clicked on is not a lensed fea...,DES-VT-JG,gri,12,DUD,Negative,2,DUD,Isolated elliptical
2,Training,Isolated elliptical,train_Most_negatives,2,pngs/Train_Most_negatives_2_1.png,pngs/Train_Most_negatives_2_2.png,pngs/Train_Most_negatives_2_3.png,pngs/Train_Most_negatives_2_4.png,166,166,...,Great job! You correctly identified that this ...,The feature you clicked on is not a lensed fea...,DES-VT-JG,gri,12,DUD,Negative,2,DUD,Isolated elliptical
3,Training,Isolated elliptical,train_Most_negatives,3,pngs/Train_Most_negatives_3_1.png,pngs/Train_Most_negatives_3_2.png,pngs/Train_Most_negatives_3_3.png,pngs/Train_Most_negatives_3_4.png,166,166,...,Great job! You correctly identified that this ...,The feature you clicked on is not a lensed fea...,DES-VT-JG,gri,12,DUD,Negative,2,DUD,Isolated elliptical
4,Training,Isolated elliptical,train_Most_negatives,4,pngs/Train_Most_negatives_4_1.png,pngs/Train_Most_negatives_4_2.png,pngs/Train_Most_negatives_4_3.png,pngs/Train_Most_negatives_4_4.png,166,166,...,Great job! You correctly identified that this ...,The feature you clicked on is not a lensed fea...,DES-VT-JG,gri,12,DUD,Negative,2,DUD,Isolated elliptical


In [8]:
names = ['Ring', 'Smooth', 'Companions', 'SDSS_Spirals', 'DES_Spirals', 'Crowded', 'Most_negatives']
#Combining previous manuscripts in one single file, not including simulations
manuscript_training = pd.DataFrame(columns = data_class.columns)

for k in range(len(names)):
    data_tmp = pd.read_csv('sample/train_' + names[k] + '.csv')
    manuscript_training = manuscript_training.append(data_tmp)
    
manuscript_training['#SUBJECT_ID'] = range(len(manuscript_training))

desired_order = ['#SUBJECT_ID', '#ID', '#CLASS', '!Subject Type', '#Type', '!Training type', 
                 '!Training SubType', '#IMAGE_NAME_1', '#IMAGE_NAME_2', '#IMAGE_NAME_3', '#IMAGE_NAME_4', 
                 '#feedback_1_x', '#feedback_1_y', '#feedback_1_tolerance', '#feedback_1_successMessage', 
                 '#feedback_1_failureMessage', '#Provenance', 'Colours', 'Size-in-arcsec', '#feedback_1_id', 
                 '#feedback_1_type', '#feedback_1_subtype']
manuscript_training = manuscript_training[desired_order]


#manuscript_training.to_csv('sample/duds_manifest.csv', index=False)
print(len(manuscript_training))
manuscript_training.head()

1152


,#SUBJECT_ID,#ID,#CLASS,!Subject Type,#Type,!Training type,!Training SubType,#IMAGE_NAME_1,#IMAGE_NAME_2,#IMAGE_NAME_3,...,#feedback_1_y,#feedback_1_tolerance,#feedback_1_successMessage,#feedback_1_failureMessage,#Provenance,Colours,Size-in-arcsec,#feedback_1_id,#feedback_1_type,#feedback_1_subtype
0,0,0,train_Ring,Training,DUD,Negative,Ring,pngs/Train_Ring_0_1.png,pngs/Train_Ring_0_2.png,pngs/Train_Ring_0_3.png,...,166,150,Ring galaxy: Well done! You correctly spotted ...,The feature you clicked on is not a lensed fea...,DES-VT-JG,gri,12,2,DUD,Ring
1,1,1,train_Ring,Training,DUD,Negative,Ring,pngs/Train_Ring_1_1.png,pngs/Train_Ring_1_2.png,pngs/Train_Ring_1_3.png,...,166,150,Ring galaxy: Well done! You correctly spotted ...,The feature you clicked on is not a lensed fea...,DES-VT-JG,gri,12,2,DUD,Ring
2,2,2,train_Ring,Training,DUD,Negative,Ring,pngs/Train_Ring_2_1.png,pngs/Train_Ring_2_2.png,pngs/Train_Ring_2_3.png,...,166,150,Ring galaxy: Well done! You correctly spotted ...,The feature you clicked on is not a lensed fea...,DES-VT-JG,gri,12,2,DUD,Ring
3,3,3,train_Ring,Training,DUD,Negative,Ring,pngs/Train_Ring_3_1.png,pngs/Train_Ring_3_2.png,pngs/Train_Ring_3_3.png,...,166,150,Ring galaxy: Well done! You correctly spotted ...,The feature you clicked on is not a lensed fea...,DES-VT-JG,gri,12,2,DUD,Ring
4,4,4,train_Ring,Training,DUD,Negative,Ring,pngs/Train_Ring_4_1.png,pngs/Train_Ring_4_2.png,pngs/Train_Ring_4_3.png,...,166,150,Ring galaxy: Well done! You correctly spotted ...,The feature you clicked on is not a lensed fea...,DES-VT-JG,gri,12,2,DUD,Ring


In [9]:
data = pd.read_csv('sample/train_' + 'Single' + '.csv')
data['#SUBJECT_ID'] = range(len(data))
data = data[desired_order]
#data.to_csv('sample/single_manifest.csv', index=False)

In [10]:
filepath = '/Users/jimenagonzalez/research/DSPL/SpaceWarps_Inspection/initial/for_test/results/'
to_compare = pd.read_csv(filepath + 'Manifest_train_betatest_update.csv')

df1, df2 = manuscript_training, to_compare
columns_in_df1_only = set(df1.columns) - set(df2.columns)
columns_in_df2_only = set(df2.columns) - set(df1.columns)

print("Columns unique to df1:", columns_in_df1_only)
print("Columns unique to df2:", columns_in_df2_only)

Columns unique to df1: {'#IMAGE_NAME_4', '#SUBJECT_ID'}
Columns unique to df2: {'#SUBJECT_NAME', '#Type Index'}


In [11]:
print(manuscript_training.columns)

Index(['#SUBJECT_ID', '#ID', '#CLASS', '!Subject Type', '#Type',
       '!Training type', '!Training SubType', '#IMAGE_NAME_1', '#IMAGE_NAME_2',
       '#IMAGE_NAME_3', '#IMAGE_NAME_4', '#feedback_1_x', '#feedback_1_y',
       '#feedback_1_tolerance', '#feedback_1_successMessage',
       '#feedback_1_failureMessage', '#Provenance', 'Colours',
       'Size-in-arcsec', '#feedback_1_id', '#feedback_1_type',
       '#feedback_1_subtype'],
      dtype='object')


In [12]:
def manuscript_pngs_real(images, data):
    feedback_tmp = feedback[feedback['Name'] == 'Real']
    data_class = pd.DataFrame()
    
    data_class['#SUBJECT_ID'] = data['COADD_OBJECT_ID']
    data_class['#ID'] = data['COADD_OBJECT_ID']
    data_class['#CLASS'] = ['train_known']*len(data)
    data_class['!Subject Type'] = ['Training']*len(data)
    data_class['#Type'] = ['LENS']*len(data)
    data_class['!Training type'] = ['Positive']*len(data)
    data_class['!Training SubType'] = [sub_types['Real']]*len(data)
    data_class['#IMAGE_NAME_1'] = data['COADD_OBJECT_ID'].apply(lambda x: f'pngs/Train_known_{x}_1.png')
    data_class['#IMAGE_NAME_2'] = data['COADD_OBJECT_ID'].apply(lambda x: f'pngs/Train_known_{x}_2.png')
    data_class['#IMAGE_NAME_3'] = data['COADD_OBJECT_ID'].apply(lambda x: f'pngs/Train_known_{x}_3.png')
    data_class['#IMAGE_NAME_4'] = data['COADD_OBJECT_ID'].apply(lambda x: f'pngs/Train_known_{x}_4.png')
    data_class['#feedback_1_x'] = [166]*len(data)
    data_class['#feedback_1_y'] = [166]*len(data)
    data_class['#feedback_1_tolerance'] = [150]*len(data)
    data_class['#feedback_1_successMessage'] = [feedback_tmp['Positive'].iloc[0]]*len(data)
    data_class['#feedback_1_failureMessage'] = [feedback_tmp['Negative'].iloc[0]]*len(data)
    data_class['#Provenance'] = ['DES-VT-JG']*len(data)
    data_class['Colours'] = ['gri']*len(data)
    data_class['Size-in-arcsec'] = [12]*len(data)
    data_class['#feedback_1_id'] = [1]*len(data)
    data_class['#feedback_1_type'] = data_class['#Type']
    data_class['#feedback_1_subtype'] = data_class['!Training SubType']
    
    data_class = data_class[desired_order]
    data_class.to_csv('sample/manifest_known'  + '.csv', index=False)
    #"""
    for i in range(len(data_class)):
        title1 = data_class['#IMAGE_NAME_1'].iloc[i]
        make_one_plot(images[i], title1, data_class, 9, 42, (0, 0, 0))
        title2 = data_class['#IMAGE_NAME_2'].iloc[i]
        make_one_plot(images[i], title2, data_class, 8, 35, (14, 2, 0))
        title3 = data_class['#IMAGE_NAME_3'].iloc[i]
        make_one_plot(images[i], title3, data_class, 10.5, 37, (14, 8, 0))
        title4 = data_class['#IMAGE_NAME_4'].iloc[i]
        make_one_plot_brit(images[i], title4, data_class)
        #make_one_plot(images[i], title3, data_class, 10.5, 37, (14, 8, 0))
    #"""
    return(data_class)


In [13]:
# Adding the train sample of known catalogs: Jacobs
filepath = '/Users/jimenagonzalez/research/DSPL/SpaceWarps_Inspection/training_sample/coadds/'
hdu_list = fits.open(filepath + 'new_ids_proposed_positive.fits')
images_real = hdu_list[1].data
data_real = Table(hdu_list[2].data).to_pandas()
hdu_list.close()

print(len(images), len(data))

192 855


In [14]:
data_mani = manuscript_pngs_real(images_real, data_real)
data_mani

,#SUBJECT_ID,#ID,#CLASS,!Subject Type,#Type,!Training type,!Training SubType,#IMAGE_NAME_1,#IMAGE_NAME_2,#IMAGE_NAME_3,...,#feedback_1_y,#feedback_1_tolerance,#feedback_1_successMessage,#feedback_1_failureMessage,#Provenance,Colours,Size-in-arcsec,#feedback_1_id,#feedback_1_type,#feedback_1_subtype
0,1425127607,1425127607,train_known,Training,LENS,Positive,Known,pngs/Train_known_1425127607_1.png,pngs/Train_known_1425127607_2.png,pngs/Train_known_1425127607_3.png,...,166,150,Congratulations!! You've spotted a gravitation...,You missed a lensed feature in this image. Hav...,DES-VT-JG,gri,12,1,LENS,Known
1,1132438466,1132438466,train_known,Training,LENS,Positive,Known,pngs/Train_known_1132438466_1.png,pngs/Train_known_1132438466_2.png,pngs/Train_known_1132438466_3.png,...,166,150,Congratulations!! You've spotted a gravitation...,You missed a lensed feature in this image. Hav...,DES-VT-JG,gri,12,1,LENS,Known
2,1561967167,1561967167,train_known,Training,LENS,Positive,Known,pngs/Train_known_1561967167_1.png,pngs/Train_known_1561967167_2.png,pngs/Train_known_1561967167_3.png,...,166,150,Congratulations!! You've spotted a gravitation...,You missed a lensed feature in this image. Hav...,DES-VT-JG,gri,12,1,LENS,Known
3,919882930,919882930,train_known,Training,LENS,Positive,Known,pngs/Train_known_919882930_1.png,pngs/Train_known_919882930_2.png,pngs/Train_known_919882930_3.png,...,166,150,Congratulations!! You've spotted a gravitation...,You missed a lensed feature in this image. Hav...,DES-VT-JG,gri,12,1,LENS,Known
4,1027961497,1027961497,train_known,Training,LENS,Positive,Known,pngs/Train_known_1027961497_1.png,pngs/Train_known_1027961497_2.png,pngs/Train_known_1027961497_3.png,...,166,150,Congratulations!! You've spotted a gravitation...,You missed a lensed feature in this image. Hav...,DES-VT-JG,gri,12,1,LENS,Known
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439,1237543091,1237543091,train_known,Training,LENS,Positive,Known,pngs/Train_known_1237543091_1.png,pngs/Train_known_1237543091_2.png,pngs/Train_known_1237543091_3.png,...,166,150,Congratulations!! You've spotted a gravitation...,You missed a lensed feature in this image. Hav...,DES-VT-JG,gri,12,1,LENS,Known
440,1042565305,1042565305,train_known,Training,LENS,Positive,Known,pngs/Train_known_1042565305_1.png,pngs/Train_known_1042565305_2.png,pngs/Train_known_1042565305_3.png,...,166,150,Congratulations!! You've spotted a gravitation...,You missed a lensed feature in this image. Hav...,DES-VT-JG,gri,12,1,LENS,Known
441,1136340852,1136340852,train_known,Training,LENS,Positive,Known,pngs/Train_known_1136340852_1.png,pngs/Train_known_1136340852_2.png,pngs/Train_known_1136340852_3.png,...,166,150,Congratulations!! You've spotted a gravitation...,You missed a lensed feature in this image. Hav...,DES-VT-JG,gri,12,1,LENS,Known
442,916018816,916018816,train_known,Training,LENS,Positive,Known,pngs/Train_known_916018816_1.png,pngs/Train_known_916018816_2.png,pngs/Train_known_916018816_3.png,...,166,150,Congratulations!! You've spotted a gravitation...,You missed a lensed feature in this image. Hav...,DES-VT-JG,gri,12,1,LENS,Known


In [15]:
filepath = '/Users/jimenagonzalez/research/DSPL/SpaceWarps_Inspection/initial/for_test/results/'
to_compare = pd.read_csv(filepath + 'Manifest_train_betatest_update.csv')

df1, df2 = manuscript_training, data_mani
columns_in_df1_only = set(df1.columns) - set(df2.columns)
columns_in_df2_only = set(df2.columns) - set(df1.columns)

print("Columns unique to df1:", columns_in_df1_only)
print("Columns unique to df2:", columns_in_df2_only)

Columns unique to df1: set()
Columns unique to df2: set()


In [16]:
# Adding the train sample of known catalogs: Jacobs
hdu_list = fits.open('real_lenses/Jacobs_train.fits')
images_jacobs = hdu_list[1].data
data_jacobs = hdu_list[2].data
data_jacobs = Table(data_jacobs)
data_jacobs = data_jacobs.to_pandas()
hdu_list.close()

data_tmp = pd.DataFrame(columns = data_save.columns)
data_tmp['new_id'] = range(0, len(images_jacobs))
data_tmp['class'] = ['Known_jacobs']*len(images_jacobs)

images_save = np.append(images_save, images_jacobs, axis = 0)
data_save = data_save.append(data_tmp)

print(len(images_jacobs))
print(len(images_save), len(data_save))

#Saving the manuscript and the PNGs
#manuscript_pngs_real(images_jacobs, data_jacobs, 'Jacobs')

152
2159 2159


In [17]:
# Adding the train sample of known catalogs: O'Donnell
hdu_list = fits.open('real_lenses/Jack_train.fits')
images_jacobs = hdu_list[1].data
data_jacobs = hdu_list[2].data
data_jacobs = Table(data_jacobs)
data_jacobs = data_jacobs.to_pandas()
hdu_list.close()

data_tmp = pd.DataFrame(columns = data_save.columns)
data_tmp['new_id'] = range(0, len(images_jacobs))
data_tmp['class'] = ['Known_jack']*len(images_jacobs)

images_save = np.append(images_save, images_jacobs, axis = 0)
data_save = data_save.append(data_tmp)

print(len(images_jacobs))
print(len(images_save), len(data_save))

#Saving the manuscript and the PNGs
#manuscript_pngs_real(images_jacobs, data_jacobs, 'Jack')

48
2207 2207


In [18]:
#write_fit_file('complete_training_sample', images_save, data_save)